# Projeto 1 - Ciência dos Dados

Nome: Leonardo Fernandes Farah Elias

Nome: Paulo Souza Chade

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [36]:
%matplotlib inline
import pandas as pd
import numpy as np
from IPython.display import display
pd.options.display.max_rows = 13
import math
import random
import re 
import sys
import functools
import operator
import emoji
import os

**Em `filename`, coloque o nome do seu arquivo de dados!**

In [37]:
filename = 'Iphone_CDados.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo Iphone_CDados.xlsx, tudo certo para prosseguir com a prova!


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [38]:
dados = pd.read_excel(filename)
dados.head(5)

,Treinamento,Filtro
0,"@kayllanefer bateria de iphone é uma merda, ja...",1
1,cabo de iphone eh feito pra estragar pqp,1
2,@johnpvx melhor iphone de sempre. não me arrep...,1
3,alguém querendo comprar um cabo/carregador de ...,1
4,to sem carregador já tava sofrendo aqui daí le...,1


In [51]:
teste = pd.read_excel(filename, sheet_name = 'Teste')
teste.tail(5)

,Teste,Filtro
69,rt @turnes_caroline: gente meu celular foi rou...,0
70,"meu iphone ja era , que odio 😑😣",0
71,"@luizzzlima_ o meu é iphone e pega, mas o da m...",0
72,alguém com iphone pode tirar print do meu fc?,0
73,amanhã não há iphone 12.😑😅 https://t.co/w6qa1q...,0


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

    Depois de algumas pesquisas, chegamos a conclusão que o trabalho seria melhor aproveitado, caso escolheriamos o Iphone como produto. Com o novo lançamento da Apple, houve uma maior repercussão na internet a respeito do celular que acabou não sendo atualizado pela marca. Dessa forma, concluimos que com um item muito presente no no dia a dia, seria uma boa oportunidade para conhecer melhor os dados. 
    Mesmo com a pandemia, o Iphone 11 foi o celular mais vendido no primeiro semestre de 2020. Portando, utilizamos o nosso algoritmo para classificar os comentários mais relevantes ou irrelevantes no twiteer. Consideramos relevantes, os comentários que sugeriam melhoras no produto (bateria, brilho, preço), críticas construtivas e também comparações feitas por usuários com outras marcas. Já os comentários irrelevantes, foram aqueles que não sugeriam nenhum conteúdo relacionado ao Iphone, como por exemplo: conversas paralelas e respostas feitas a usuários desconhecidos (retweet). 

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [40]:
# Função de limpeza muito simples que troca alguns sinais básicos por espaços.
def cleanup(text):
    punctuation = '[!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

# Função de limpeza muito simples que remove o arroba.
def remove_arroba(lista):
    arroba = "@"
    for i in lista: 
        if arroba in i:
            lista.remove(i)
    return lista

# Função de limpeza muito simples que remove as \ e links.
def remove_link(lista):
    link = "/"
    for i in lista: 
        if link in i:
            lista.remove(i)
    return lista

In [41]:
# Filtros para Relevante e Irrelevante.
Relevante = dados["Filtro"].isin(["1"])
Irrelevante = dados["Filtro"].isin(["0"])
# Criando data frame com filtro.
Relevante = dados.loc[Relevante, :]
Irrelevante = dados.loc[Irrelevante, :]
# Variáveis que serão utilizadas na função para criar uma lista.
Treino_Relevante = Relevante["Treinamento"]        # Variável com palavras Relevantes.
Treino_Irrelevante = Irrelevante["Treinamento"]    # Variável com palavras Irrelevantes.

# Número de frases classificadas como Relevantes ou Irrelevantes (.count).
total_frases_relevantes = Relevante.count()[0]
total_frases_irrelevantes = Irrelevante.count()[0]

In [42]:
# Função criada para transformar a coluna no data frame com palavras Relevantes e Irrelevantes em lista.
def make_list(Treino_Relevante):
    treinamento_relevante = []
    # Fazendo a lista.
    for i in Treino_Relevante:
        treinamento_relevante.append(i)

    coma = "'"
    # Transformar em texto.
    treinamento_relevante = coma.join(treinamento_relevante)
    # Função que limpa (já criada em cima).
    limpa = cleanup(treinamento_relevante)
    # Transforma em letras minúsculas.
    minusculo = limpa.lower()
    palavras_relevantes = minusculo.split()
    
    # Removendo/separando emoji.
    lista_nova = []
    for i in palavras_relevantes: 
        em_split_emoji = emoji.get_emoji_regexp().split(i)
        em_split_whitespace = [o.split() for o in em_split_emoji]
        em_split = functools.reduce(operator.concat, em_split_whitespace)
        lista_nova.append(em_split)
    
    lista_emoji = []
    for w in lista_nova:
        for j in w:
            lista_emoji.append(j)
    
    # Usando funções que ja foram criadas.
    lista = remove_arroba(lista_emoji)
    lista = remove_link(lista)
    return lista

# Transformando em lista por meio da função make_list.
todas_palavras_relevantes = make_list(Treino_Relevante)
todas_palavras_irrelevantes = make_list(Treino_Irrelevante) 

In [43]:
# pdSeries.
serie_relevante = pd.Series(todas_palavras_relevantes)
serie_irrelevante = pd.Series(todas_palavras_irrelevantes)

# Criando tabelas de frequencia relativas.
tabela_relevantes = serie_relevante.value_counts()
tabela_irrelevantes = serie_irrelevante.value_counts()
Nr = sum(tabela_relevantes)
Ni = sum(tabela_irrelevantes)

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [44]:
def limpa(tweet):
    pri = cleanup(tweet)
    return pri
# Criaremos duas funções para ambas as probabilidades. 
# Função da probabilidade de ser frase dado que ela é Relevante P(frase/relevante).
def P_tweet_dado_relevante(lista): 
    prob_tweet_dado_relevante = 0
    alpha = 1
    V = 100000
    prob_tweets = []
    for tweet in lista:
        prob_tweet_dado_relevante = 0
        tweet_minusculo = tweet.lower()
        tweet_limpo = limpa(tweet_minusculo) 
        lista_limpo = tweet_limpo.split()
        
        # Removendo/separando emoji.
        lista_nova = []
        for i in lista_limpo: 
            em_split_emoji = emoji.get_emoji_regexp().split(i)
            em_split_whitespace = [o.split() for o in em_split_emoji]
            em_split = functools.reduce(operator.concat, em_split_whitespace)
            lista_nova.append(em_split)
        
        # Organizando os emojis na lista.
        lista_emoji = []
        for w in lista_nova:
            for j in w:
                lista_emoji.append(j)
        
        lista_limpo = remove_arroba(lista_emoji)
        lista_limpo = remove_link(lista_limpo)
        
        # Calculando a probabilidade.
        for palavra in lista_limpo:
            if palavra in tabela_relevantes:
                prob_palavra_dado_relevante = (tabela_relevantes[palavra] + alpha)/ (Nr+(V * alpha))
            else:
                prob_palavra_dado_relevante = alpha/(Nr + (alpha * V))
            
            # Ao invés de utilizarmos um produtório fizemos um somatório com log.
            prob_tweet_dado_relevante += math.log(prob_palavra_dado_relevante)
        
        prob_tweets.append(prob_tweet_dado_relevante)
        
    return prob_tweets

In [45]:
# Função da probabilidade de ser frase dado que ela é Irrelevante P(frase/irrelevante).
def P_tweet_dado_irrelevante(lista):
    prob_tweet_dado_irrelevante = 0
    alpha = 1
    V = 100000
    prob_tweets = []
    for tweet in lista:
        prob_tweet_dado_irrelevante = 0
        tweet_minusculo = tweet.lower()
        tweet_limpo = limpa(tweet_minusculo)
        lista_limpo = tweet_limpo.split()
        
        # Removendo/separando emoji.
        lista_nova = []
        for i in lista_limpo: 
            em_split_emoji = emoji.get_emoji_regexp().split(i)
            em_split_whitespace = [o.split() for o in em_split_emoji]
            em_split = functools.reduce(operator.concat, em_split_whitespace)
            lista_nova.append(em_split)
        
        # Organizando os emojis na lista.
        lista_emoji = []
        for w in lista_nova:
            for j in w:
                lista_emoji.append(j)
        
        lista_limpo = remove_arroba(lista_emoji)
        lista_limpo = remove_link(lista_limpo)
             
        # Calculando a probabilidade.
        for palavra in lista_limpo:
            if palavra in tabela_irrelevantes:
                prob_palavra_dado_irrelevante = (tabela_irrelevantes[palavra] + alpha)/ (Ni+(V * alpha))
            else:
                prob_palavra_dado_irrelevante = alpha/(Ni + (alpha * V))
                
            # Ao invés de utilizarmos um produtório fizemos um somatório com log.
            prob_tweet_dado_irrelevante += math.log(prob_palavra_dado_irrelevante)
        
        prob_tweets.append(prob_tweet_dado_irrelevante)
        
    return prob_tweets

In [46]:
# Calculando as probabilidades.
# Probabilidade de ser Relevante.
P_R = total_frases_relevantes/(total_frases_relevantes+total_frases_irrelevantes)
# Probabilidade de ser Irrelevante.
P_I = 1 - P_R

In [47]:
teste_relevante = teste["Filtro"].isin(["1"])
teste_irrelevante = teste["Filtro"].isin(["0"])
Teste_Relevante = teste.loc[teste_relevante, :]
Teste_Irrelevante = teste.loc[teste_irrelevante, :]

def lista_treino(data_frame):    
    lista_teste = []
    for i in data_frame["Teste"]:
        lista_teste.append(i)

    return lista_teste

lista_teste = lista_treino(teste)

In [48]:
# Lista com valores negativos.
Prob_tweet_dado_relevante = P_tweet_dado_relevante(lista_teste)
Prob_tweet_dado_irrelevante = P_tweet_dado_irrelevante(lista_teste)

Prob_relevante_dado_tweet = [i + math.log(P_R) for i in Prob_tweet_dado_relevante]
Prob_irrelevante_dado_tweet = [i + math.log(P_I) for i in Prob_tweet_dado_irrelevante]

In [49]:
# For que transforma a lista com valores de 0 e 1 (Irrelevante e Relevante, respectivamente).
valor_maior = []
for i in range(len(Prob_relevante_dado_tweet)):
    if Prob_relevante_dado_tweet[i] > Prob_irrelevante_dado_tweet[i]:
        valor_maior.append(1)
    else:
        valor_maior.append(0)

#sum(valor_maior)

11

In [50]:
# Tabela cruzada para ver indíces de acertos, qualidade do nosso algoritmo.
teste["Predicted"] = valor_maior
x = (100 * pd.crosstab(teste.Filtro, teste.Predicted, normalize = "index")).round(decimals=2)
x

Predicted,0,1
Filtro,,
0,90.24,9.76
1,78.79,21.21


___
### Concluindo

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**